# Attention with hidden state vectors

In [ ]:
import torch

In [ ]:
seq_len = 4
hidden_dim = 3

In [ ]:
x = torch.randn(seq_len, hidden_dim)
print(x.shape)
x

In [ ]:
x = x.repeat(seq_len, 1, 1)
print(x.shape)

In [ ]:
mask = torch.triu(torch.ones(x.shape[-1], x.shape[0], x.shape[0]))
mask

In [ ]:
masked = torch.mul(mask, x.transpose(0, -1))
print(masked.shape)
masked.transpose(0, -1)

In [ ]:
attention_dim = 2
w_a = torch.randn(hidden_dim, attention_dim)
tanh = torch.nn.Tanh()
key = tanh(torch.matmul(masked.transpose(0, -1), w_a))
print(key.shape)
key

In [ ]:
q_dim = 1
softmax = torch.nn.Softmax(dim=-1)
q = torch.randn(q_dim, attention_dim)
temp = torch.matmul(q, key.transpose(1, 2))
print(temp.shape)
temp

In [ ]:
softmax_mask = ((torch.tril(torch.ones(temp.shape[0], temp.shape[0]), diagonal=0) != 1) * -float("inf")).nan_to_num(
    nan=0
)
softmax_mask = softmax_mask.unsqueeze(1)
softmax_mask

In [ ]:
temp = temp + softmax_mask
temp

In [ ]:
d = softmax(temp)
print(d.shape)
d.squeeze()

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(d.squeeze())
plt.show()

In [ ]:
a = torch.matmul(d, masked.unsqueeze(-1)).transpose(0, 1)
print(a.shape)
a.squeeze()